# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [53]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


/home/nika/.local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['permutation', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [54]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Belly', 'Back', 'Left', 'Frog', 'Stand', 'HeadBack', 'Crouch', 'Sit', 'StandInit', 'Right', 'Knee']


In [55]:
def load_pose_data(i):
    '''load pose data from file'''

    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    file = open(filename, 'rb')
    data = pickle.load(file)
    file.close()
    target = [i] * len(data)
    return data, target

In [56]:
# load all the data
all_data = []
all_target = []

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print('total number of data', len(all_data))

total number of data 222


In [57]:
# shuffle data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

print(permutation)

[ 81  26 158 145 164 150 108 163  88 151 216   3 144  14  96 146 206 214
 193 175 107 104  77  90 118  55  10 148 102 170  44 220 129  31  37 179
  53  78 100 111  65  18 113 153 200 178  43 117  67 132  75  41 154  46
 162   2 190  38 103 133  25 173 141 196  98 155 215 188  52  82 121  20
  35  23  22 211  33 210 181  74   0  16  28  19 127 149 126 138  70 189
  34 105 106  93 177  61  95  11 114  56 201   4   8  94 109 195  42 198
 142 209 139 168 136 130 169  47 197 112 156  72 187  21 124 204  66  62
 191 123  87 176 101  63  30 167 172  58 165 131 182 199 125 161  54 152
 147  24  91 221   1  39 212  99 194 202  29  69  36 186  50   5  97 110
 180  76  49  13  32  92   7 185 208 217 115  48  57  86  73 140 134 174
 128  89  45 160 143  51 207 203 218  27  12 171 122 183 137 116  80 213
  15 205 119 120  17 219 184 159   6  71  59  68  83  79  60 192  40  85
 166  84   9  64 157 135]


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [58]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [59]:
dataset = []
labelset = []

for i in permutation:
    dataset.append(all_data[i])
    labelset.append(all_target[i])

clf.fit(dataset[:n_training_data], labelset[:n_training_data])

SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### predicting

In [60]:
#made prediciton on full dataset, not sure what was here expected?
predicted = clf.predict(dataset)

In [61]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [62]:
expected = labelset
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        23
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        11
           5       1.00      1.00      1.00        10
           6       0.97      1.00      0.98        30
           7       1.00      1.00      1.00        26
           8       1.00      1.00      1.00        52
           9       1.00      0.91      0.95        11
          10       1.00      1.00      1.00        10

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[19  0  0  0  0  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [63]:
predicted = clf.predict(dataset[n_training_data:])
expected = labelset[n_training_data:]

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00        12
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         4

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [64]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
file = open(ROBOT_POSE_CLF, 'wb')
pickle.dump(clf, file)
file.close()

Then, in the application we can load the trained classifier again.

In [66]:
file  = open(ROBOT_POSE_CLF, 'rb')
clf2 = pickle.load(file)
clf2.predict(dataset[n_training_data:]), labelset[n_training_data:]

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00        12
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         4

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 